### Import packages

In [1]:
# Add python code here

# Make your own: Accelerating Wall Reactor

Here we will solve an ignition problem where the normal reactor governing equations are
extended with additional equations implemented in Python.
This demonstrates an approach for solving problems where Cantera's built-in
reactor models are not sufficient for describing the system in question. Unlike
the 'custom.py' example, in this example Cantera's existing Reactor and
ReactorNet code is still used, with only the modifications to the standard
equations implemented in Python by extending the ExtensibleReactor class.
Wall objects in Cantera are normally massless, with the velocity either imposed
or proportional to the pressure difference. Here, we simulate a wall where the
acceleration is proportional to the pressure difference, and the velocity is
determined by integrating the equation of motion. This requires adding a new
variable to the reactor's state vector which represents the wall velocity.

### Unmodified Perfectly Stirred Reactor
The perfectly stirred reactor (aka "PSR") is defined by a relatively simple set of governing equations, providing an excellent opportunity to demonstrate how we can use Cantera as a "calculator" to provide the terms and variables required for solving a set of equations of our own choosing.

<img src="PSR.png" alt="Cartoon of a Perfectly Stirred Chemical Reactor." style="width: 500px;"/>

The model includes user inputs for the inlet gas conditions (temperature, pressure, composition) and mass flux (typically defined in terms of the residence time $\tau$), some condition for the outlet flow, and an optional heat transfer boundary condition.

## Differential equations:

An Ideal Gas Reactor is defined by the four state variables: 

- m, the mass of the reactor's contents (in kg)

- V, the reactor volume (in m^3)

- T, the temperature (in K)

- Y_k, the mass fractions for each species (dimensionless)

The total mass of the reactor's contents changes as a result of flow through
the reactor's inlets and outlets, and production of homogeneous phase species
on the reactor walls:


1) 

$\frac{dm}{dt} = \sum_{in} \dot{m}_{in} - \sum_{out} \dot{m}_{out} +\dot{m}_{wall}$

Where the subscripts *in* and *out* refer to the sum of the superscipted property
over all inlets and outlets respectively. A dot above a variable signifies a time 
derivative.

The reactor volume changes as a function of time due to the motion of one or
more walls:

2) 

$\frac{dV}{dt} = \sum_w f_w A_w v_w(t)$

Where $f_w = \pm 1$ indicates the facing of the wall (whether moving the wall increases or
decreases the volume of the reactor), $A_w$ is the
surface area of the wall, and $v_w(t)$ is the velocity of the wall as a
function of time.

In case of the Ideal Gas Reactor Model, the reactor temperature $T$ is
used instead of the total internal energy $U$ as a state variable. For an
ideal gas, we can rewrite the total internal energy in terms of the mass
fractions and temperature:

   $U = m \sum_k Y_k u_k(T)$

   $\frac{dU}{dt} = u \frac{dm}{dt} + m c_v \frac{dT}{dt}+ m \sum_k u_k \frac{dY_k}{dt}$

Substituting the corresponding derivatives yields an equation for the
temperature:

3)  

$m c_v \frac{dT}{dt} = - p \frac{dV}{dt} + \dot{Q}+ \sum_{in} \dot{m}_{in} \left( h_{in} - \sum_k u_k Y_{k,in} \right)- \frac{p V}{m} \sum_{out} \dot{m}_{out} - \sum_k \dot{m}_{k,gen} u_k$

While this form of the energy equation is somewhat more complicated, it
significantly reduces the cost of evaluating the system Jacobian, since the
derivatives of the species equations are taken at constant temperature instead
of constant internal energy.

The rate at which species k is generated through homogeneous phase
reactions is $V \dot{\omega}_k W_k$, and the total rate at which species
k is generated is:

   $\dot{m}_{k,gen} = V \dot{\omega}_k W_k + \dot{m}_{k,wall}$

The rate of change in the mass of each species is:

  $\frac{d(mY_k)}{dt} = \sum_{in} \dot{m}_{in} Y_{k,in} - \sum_{out} \dot{m}_{out} Y_k + \dot{m}_{k,gen}$

Expanding the derivative on the left hand side and substituting the equation
for $dm/dt$, the equation for each homogeneous phase species is:

4) 

$m \frac{dY_k}{dt} = \sum_{in} \dot{m}_{in} (Y_{k,in} - Y_k)+\dot{m}_{k,gen} - Y_k \dot{m}_{wall} $

Equations 1-4 are the governing equations for an Ideal Gas Reactor.

### Creating a New Reactor Class Type:

In [2]:
# Add python code here

### Instantiate Cantera objects:

In [3]:
# Add python code here

### Initialize the Solution Vector and compute solution:

In [4]:
# Add python code here

### Some basic plotting:

In [5]:
# Add python code here